<a href="https://colab.research.google.com/github/eoinleen/protein-design-final-dir/blob/main/Str_analysis_PAE_plotting_combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
RFdiffusion Structure Analysis and Visualization Tool
==================================================

About
-----
This script combines RFdiffusion pathway structure analysis with AF2 score visualization
into a single automated pipeline. It processes PDB structures and generates publication-ready
visualizations of structure-function relationships.

Performance Note
--------------
Processing speed is approximately 2 seconds per structure due to comprehensive structural analysis.

What Does It Do?
--------------
1. Analyzes PDB files from RFdiffusion-MPNN-AF2 pipeline
2. Calculates key structural parameters:
   - Buried surface area
   - Hydrogen bonds
   - Hydrophobic contacts
   - Salt bridges
3. Combines structural data with AF2 scores
4. Generates publication-ready plots in PowerPoint format including:
   - Structure-function correlation plots
   - Comprehensive iPAE score visualization

Credits
-------
Original Analysis Code: Dr. Eoin Leen, University of Leeds
Visualization Components: Claude AI
Integration & Optimization: Claude AI & User
Version: 1.0 (2025)
"""

# Install required packages
!pip install -q biopython pandas freesasa numpy matplotlib seaborn python-pptx plotly kaleido

# Import required libraries
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files, drive
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Any
from Bio import PDB
from Bio.PDB.PDBIO import PDBIO
from Bio.PDB.Polypeptide import is_aa
from Bio.PDB.Structure import Structure
import freesasa
import numpy as np
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches, Cm
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class StructureValidationError(Exception):
    pass

def validate_pdb_file(file_path: str) -> bool:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"PDB file not found: {file_path}")
    try:
        with open(file_path, 'r') as f:
            first_line = f.readline()
            if not any(marker in first_line for marker in ['HEADER', 'ATOM', 'MODEL']):
                raise StructureValidationError(f"Invalid PDB: {file_path}")
    except UnicodeDecodeError:
        raise StructureValidationError(f"Not a valid text file: {file_path}")
    return True

def safe_structure_load(parser: PDB.PDBParser, file_path: str) -> Optional[Structure]:
    try:
        validate_pdb_file(file_path)
        structure = parser.get_structure('protein', file_path)
        if not list(structure.get_models()):
            raise StructureValidationError("No models")
        if not list(list(structure.get_models())[0].get_chains()):
            raise StructureValidationError("No chains")
        return structure
    except Exception as e:
        print(f"Error loading {file_path}: {str(e)}")
        return None

def calculate_buried_surface_area(pdb_file: str) -> Tuple[Optional[float], Optional[Dict[str, float]]]:
    parser = PDB.PDBParser(QUIET=True)
    structure = safe_structure_load(parser, pdb_file)
    if not structure:
        return None, None
    try:
        chains = list(structure.get_chains())
        if len(chains) < 2:
            print(f"Warning: {pdb_file} has fewer than 2 chains")
            return None, None

        combined_structure = freesasa.Structure(pdb_file)
        result = freesasa.calc(combined_structure)
        total_area = result.totalArea()

        chain_areas = {}
        io = PDBIO()
        temp_files = []

        for chain in chains:
            new_structure = PDB.Structure.Structure('temp')
            new_model = PDB.Model.Model(0)
            new_structure.add(new_model)
            new_model.add(chain.copy())

            temp_file = f"temp_chain_{chain.id}.pdb"
            temp_files.append(temp_file)

            io.set_structure(new_structure)
            io.save(temp_file)

            chain_structure = freesasa.Structure(temp_file)
            chain_result = freesasa.calc(chain_structure)
            chain_areas[chain.id] = chain_result.totalArea()

        for temp_file in temp_files:
            if os.path.exists(temp_file):
                os.remove(temp_file)

        total_individual_area = sum(chain_areas.values())
        buried_surface_area = abs(total_individual_area - total_area) / 2
        return buried_surface_area, chain_areas

    except Exception as e:
        print(f"Error calculating BSA for {pdb_file}: {str(e)}")
        return None, None

def calculate_hydrogen_bonds(structure: Structure) -> int:
    try:
        h_bonds = 0
        for chain1 in structure.get_chains():
            for chain2 in structure.get_chains():
                if chain1.id >= chain2.id:
                    continue
                for res1 in chain1.get_residues():
                    if not is_aa(res1):
                        continue
                    for res2 in chain2.get_residues():
                        if not is_aa(res2):
                            continue
                        if 'O' in res1 and 'N' in res2:
                            distance = res1['O'] - res2['N']
                            if distance < 3.5:
                                h_bonds += 1
        return h_bonds
    except Exception as e:
        print(f"Error calculating H-bonds: {str(e)}")
        return 0

def calculate_hydrophobic_contacts(structure: Structure) -> int:
    try:
        hydrophobic_residues = {'ALA', 'VAL', 'LEU', 'ILE', 'MET', 'PHE', 'TRP', 'PRO'}
        contacts = 0
        for chain1 in structure.get_chains():
            for chain2 in structure.get_chains():
                if chain1.id >= chain2.id:
                    continue
                for res1 in chain1.get_residues():
                    if not is_aa(res1) or res1.get_resname() not in hydrophobic_residues:
                        continue
                    for res2 in chain2.get_residues():
                        if not is_aa(res2) or res2.get_resname() not in hydrophobic_residues:
                            continue
                        min_distance = float('inf')
                        for atom1 in res1.get_atoms():
                            for atom2 in res2.get_atoms():
                                distance = atom1 - atom2
                                min_distance = min(min_distance, distance)
                        if min_distance < 5.0:
                            contacts += 1
        return contacts
    except Exception as e:
        print(f"Error calculating hydrophobic contacts: {str(e)}")
        return 0

def calculate_salt_bridges(structure: Structure) -> int:
    try:
        acidic = {'ASP', 'GLU'}
        basic = {'LYS', 'ARG', 'HIS'}
        salt_bridges = 0
        for chain1 in structure.get_chains():
            for chain2 in structure.get_chains():
                if chain1.id >= chain2.id:
                    continue
                for res1 in chain1.get_residues():
                    if not is_aa(res1):
                        continue
                    res1_name = res1.get_resname()
                    for res2 in chain2.get_residues():
                        if not is_aa(res2):
                            continue
                        res2_name = res2.get_resname()
                        if ((res1_name in acidic and res2_name in basic) or
                            (res1_name in basic and res2_name in acidic)):
                            min_distance = float('inf')
                            for atom1 in res1.get_atoms():
                                for atom2 in res2.get_atoms():
                                    distance = atom1 - atom2
                                    min_distance = min(min_distance, distance)
                            if min_distance < 4.0:
                                salt_bridges += 1
        return salt_bridges
    except Exception as e:
        print(f"Error calculating salt bridges: {str(e)}")
        return 0

def save_results_as_df(results: List[Dict[str, Any]], pdb_directory: str) -> pd.DataFrame:
    analysis_data = []
    for result in results:
        filename = result['file_name'].replace('.pdb', '')
        try:
            design_num = int(filename.split('design')[1].split('_')[0])
            variant_num = int(filename.split('_n')[1])
            analysis_data.append({
                'design': design_num,
                'n': variant_num,
                'buried_surface_area': result['buried_surface_area'] if result['buried_surface_area'] else 0,
                'hydrogen_bonds': result['hydrogen_bonds'],
                'hydrophobic_contacts': result['hydrophobic_contacts'],
                'salt_bridges': result['salt_bridges']
            })
        except Exception as e:
            print(f"Error parsing filename {filename}: {str(e)}")
            continue

    df = pd.DataFrame(analysis_data)
    df = df.sort_values(['design', 'n']).reset_index(drop=True)
    output_file = os.path.join(pdb_directory, "structure_analysis.csv")
    df.to_csv(output_file, index=False)
    print(f"Saved structure analysis to {output_file}")
    return df

def merge_with_af2_scores(structure_df: pd.DataFrame, af2_scores_file: str) -> pd.DataFrame:
    af2_df = pd.read_csv(af2_scores_file)
    merged_df = pd.merge(af2_df, structure_df, on=['design', 'n'], how='left')
    merged_df = merged_df.sort_values(['design', 'n']).reset_index(drop=True)
    return merged_df

def create_pptx_plots(df: pd.DataFrame, output_dir: str):
    prs = Presentation()
    prs.slide_width = Cm(21)
    prs.slide_height = Cm(29.7)

    # First slide - correlation plots
    slide1 = prs.slides.add_slide(prs.slide_layouts[5])

    fig, axes = plt.subplots(3, 2, figsize=(8.27, 11.69))
    axes = axes.flatten()

    y_vars = ['i_ptm', 'rmsd', 'buried_surface_area',
              'hydrogen_bonds', 'hydrophobic_contacts', 'salt_bridges']
    titles = ['iPTM', 'RMSD (Å)', 'Buried Surface Area (Å²)',
             '# of Hydrogen Bonds', '# of Hydrophobic Contacts', '# of Salt Bridges']

    for ax, y_var, title in zip(axes, y_vars, titles):
        sns.scatterplot(data=df, x='i_pae', y=y_var, ax=ax, color='black', marker='x', s=16)
        ax.set_xlabel('i_PAE')
        ax.set_ylabel(title)
        ax.set_title(title)
        ax.set_facecolor('white')

    fig.patch.set_facecolor('white')
    plt.tight_layout()

    temp_img1 = os.path.join(output_dir, 'temp_plots1.png')
    plt.savefig(temp_img1, bbox_inches='tight', dpi=300, facecolor='white')
    plt.close()

    left = Cm(2)
    top = Cm(2)
    slide1.shapes.add_picture(temp_img1, left, top)

    # Second slide - iPAE visualization
    slide2 = prs.slides.add_slide(prs.slide_layouts[5])

    # Create iPAE visualization
    fig = make_subplots(
        rows=4,
        cols=1,
        vertical_spacing=0.08,
        subplot_titles=[f"Designs {i*8}-{(i+1)*8-1}" for i in range(4)]
    )

    rows_per_subplot = 512  # 8 designs × 64 sequences = 512 rows per subplot
    colors = ['black', 'red']

    for i in range(4):
        start_idx = i * rows_per_subplot
        end_idx = start_idx + rows_per_subplot
        chunk = df.iloc[start_idx:end_idx].copy()

        for design_num in chunk['design'].unique():
            mask = chunk['design'] == design_num
            color = colors[design_num % 2]

            fig.add_trace(
                go.Bar(
                    x=chunk[mask].index,
                    y=chunk[mask]['i_pae'],
                    showlegend=False,
                    marker_color=color,
                    width=1,
                ),
                row=i+1,
                col=1
            )

        fig.update_yaxes(
            range=[0, 30],
            title_text='iPAE' if i == 1 else None,
            row=i+1,
            col=1
        )

        design_numbers = sorted(chunk['design'].unique())
        fig.update_xaxes(
            tickmode='array',
            ticktext=design_numbers,
            tickvals=[start_idx + (j*64) + 32 for j in range(len(design_numbers))],
            row=i+1,
            col=1,
            title_text='Design Number' if i == 3 else None
        )

    fig.update_layout(
        title='iPAE Scores by Design Number and Sequence (Scale: 0-30)',
        height=1000,
        width=1200,
        showlegend=False,
        margin=dict(t=50, b=50, r=150, l=50),
        paper_bgcolor='white',
        plot_bgcolor='white'
    )

    temp_img2 = os.path.join(output_dir, 'temp_plots2.png')
    fig.write_image(temp_img2)

    left = Cm(1)
    top = Cm(1)
    slide2.shapes.add_picture(temp_img2, left, top)

    # Save PowerPoint and clean up temporary files
    pptx_path = os.path.join(output_dir, 'protein_analysis_plots.pptx')
    prs.save(pptx_path)
    os.remove(temp_img1)
    os.remove(temp_img2)
    print(f"Saved PowerPoint to {pptx_path}")

def process_multiple_pdb_files(pdb_directory: str, af2_scores_file: str = None) -> pd.DataFrame:
    if not os.path.exists(pdb_directory):
        raise FileNotFoundError(f"Directory not found: {pdb_directory}")

    results = []
    parser = PDB.PDBParser(QUIET=True)
    pdb_files = [f for f in os.listdir(pdb_directory) if f.endswith('.pdb')]

    if not pdb_files:
        print(f"No PDB files found in {pdb_directory}")
        return pd.DataFrame()

    print(f"Processing {len(pdb_files)} PDB files...")
    for file_name in pdb_files:
        pdb_file = os.path.join(pdb_directory, file_name)
        print(f"Processing {file_name}...")

        structure = safe_structure_load(parser, pdb_file)
        if not structure:
            continue

        buried_surface_area, chain_areas = calculate_buried_surface_area(pdb_file)
        h_bonds = calculate_hydrogen_bonds(structure)
        hydrophobic = calculate_hydrophobic_contacts(structure)
        salt_bridges = calculate_salt_bridges(structure)

        results.append({
            'file_name': file_name,
            'buried_surface_area': buried_surface_area,
            'hydrogen_bonds': h_bonds,
            'hydrophobic_contacts': hydrophobic,
            'salt_bridges': salt_bridges,
            'chain_areas': chain_areas
        })

    structure_df = save_results_as_df(results, pdb_directory)

    if af2_scores_file and os.path.exists(af2_scores_file):
        print(f"Merging with AF2 scores from {af2_scores_file}")
        final_df = merge_with_af2_scores(structure_df, af2_scores_file)
        combined_file = os.path.join(pdb_directory, "combined_analysis.csv")
        final_df.to_csv(combined_file, index=False)
        print(f"Saved combined results to {combined_file}")
        create_pptx_plots(final_df, pdb_directory)
        return final_df
    return structure_df

# Main execution
drive.mount('/content/drive')

pdb_directory = '/content/drive/MyDrive/PDB-files/202501xx/3NOB-70-110-all_pdb'  # Change this path
af2_scores_path = os.path.join(pdb_directory, 'af2_scores.csv')

if not os.path.exists(af2_scores_path):
    af2_scores_path = None
    print("No AF2 scores file found - will generate structure analysis only")

print("Starting analysis...")
print(f"Processing PDB files from: {pdb_directory}")

results_df = process_multiple_pdb_files(pdb_directory, af2_scores_path)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.1/270.1 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 8.4 MB/s eta 0:00:00
